In [31]:
import re
import requests
import pandas as pd

from plotly import express as px, figure_factory as ff

ACP_GH_ORG = "avalanche-foundation"
ACP_GH_REPO = "ACPs"
AVALANCHE_API_URL = "https://api.avax.network"

In [32]:
# Get the list of ACPs
url = "https://api.github.com/repos/{}/{}/contents/ACPs".format(ACP_GH_ORG, ACP_GH_REPO)
asp_files_res = requests.get(url)

acp_files = asp_files_res.json()
acps = []

for acp in acp_files:
    if acp["name"] == "TEMPLATE.md":
        continue

    # Get the Markdown of the ACP
    acp_markdown = requests.get(acp["download_url"]).text
    acp_info = re.search(
        r"```text\nACP: ?(.*)\nTitle: ?(.+)\nAuthor\(s\): ?([^\<]*)<.*>\nDiscussions-To: ?(.*)\nStatus: ?(.*)\nTrack: ?(.*)",
        acp_markdown,
    )

    acps.append(
        {
            "acp": acp_info.group(1),
            "title": acp_info.group(2),
            "authors": acp_info.group(3),
            "discussions_to": acp_info.group(4),
            "status": acp_info.group(5),
            "track": acp_info.group(6),
        }
    )

acps_df = (
    pd.DataFrame(acps)
    # Make sure the ACP column is a string
    .astype({"acp": str})
    .drop(columns=["discussions_to"])
)

In [33]:
# ACPs table
acps_table = ff.create_table(acps_df)
acps_table.update_layout(
	title_text="Avalanche Community Proposals",
	title_x=0.5,
	margin=dict(l=0, r=0, t=30, b=0),
)
# Use a dark theme
acps_table.layout.template = "plotly_dark"

acps_table.show()

In [34]:
# ACPs by status bar chart
acps_by_status = acps_df.groupby("status").count().reset_index()

acps_by_status = px.bar(
	acps_by_status,
	x="status",
	y="acp",
	title="ACPs by Status",
	labels={"acp": "Number of ACPs", "status": "Status"},
	color="status",
	template="plotly_dark",
)

acps_by_status.show()

In [35]:
# Get Avalanche network peers list
data = {
	"jsonrpc": "2.0",
	"id": 1,
	"method": "info.peers",
	"params": {},
}
avalanche_peers_res = requests.post(f"{AVALANCHE_API_URL}/ext/info", json=data)

avalanche_peers = avalanche_peers_res.json()["result"]["peers"]

peers_df = pd.DataFrame(avalanche_peers)

In [36]:
# Get the current Avalanche network validators list
data = {
	"jsonrpc": "2.0",
	"id": 1,
	"method": "platform.getCurrentValidators",
	"params": {},
}
current_validators_res = requests.post(f"{AVALANCHE_API_URL}/ext/P", json=data)

curr_validators = current_validators_res.json()["result"]["validators"]
curr_validators_df = pd.DataFrame(curr_validators)

In [37]:
# Get the total stake of all validators
data = {
	"jsonrpc": "2.0",
	"id": 1,
	"method": "platform.getTotalStake",
	"params": {},
}
avalanche_total_stake_res = requests.post(f"{AVALANCHE_API_URL}/ext/P", json=data)

avalanche_total_stake = avalanche_total_stake_res.json()["result"]["stake"]

In [38]:
# Enrich the list of peers with the validator stake and the delegator stake (= weight) (join on Node ID)
validators_df = peers_df.merge(curr_validators_df[["nodeID", "stakeAmount", "delegatorWeight"]], on="nodeID", how="left")
# Cast the stakeAmount and delegatorWeight columns to numeric and divide by 10^9 to get the stake in AVAX
validators_df["stakeAmount"] = pd.to_numeric(validators_df["stakeAmount"]) / 10 ** 9
validators_df["delegatorWeight"] = pd.to_numeric(validators_df["delegatorWeight"]) / 10 ** 9
# Calculate the total stake of the validator (stakeAmount + delegatorWeight)
validators_df["totalStakeAmount"] = validators_df["stakeAmount"] + validators_df["delegatorWeight"]

In [39]:
# For each ACP, get the list of validators that support or object it
# Each validator has a supportedACPs and objectedACPs list

# Explode the list of validators by supportedACPs
validators_acp_sup_df = (
	validators_df
	.explode("supportedACPs")
	# Group by supportedACPs and sum the totalStakeAmount
	.groupby("supportedACPs").sum().reset_index()
	# Rename the supportedACPs column to acp
	.rename(columns={"supportedACPs": "acp"})
	# Make sure the acp column is a string
	.astype({"acp": str})
)
# Explode the list of validators by objectedACPs
validators_acp_obj_df = (
	validators_df
	.explode("objectedACPs")
	# Group by objectedACPs and sum the totalStakeAmount
	.groupby("objectedACPs").sum().reset_index()
	# Rename the objectedACPs column to acp
	.rename(columns={"objectedACPs": "acp"})
	# Make sure the acp column is a string
	.astype({"acp": str})
)

In [40]:
# For each ACP, get the number of validators and the total stake that support or object it
acps_sup_df = (
    acps_df.merge(validators_acp_sup_df, on="acp", how="left")
    .groupby("acp", as_index=False)
    .agg({"totalStakeAmount": "sum", "nodeID": "count"})
    .rename(columns={"nodeID": "supportersCount", "totalStakeAmount": "supportersTotalStake"})
)
acps_obj_df = (
    acps_df.merge(validators_acp_obj_df, on="acp", how="left")
    .groupby("acp", as_index=False)
    .agg({"totalStakeAmount": "sum", "nodeID": "count"})
    .rename(columns={"nodeID": "objectorsCount", "totalStakeAmount": "objectorsTotalStake"})
)
acps_support_df = acps_sup_df.merge(acps_obj_df, on="acp", how="left")

In [41]:
# Enrich the ACPs table with:
# - the total number of validators in the network
# - the total stake of all validators
# - the percentage of validators that support or object the ACP
# - the percentage of stake that supports or objects the ACP
# - the percentage of votes that support or object the ACP
acps_support_df["networkValidatorsCount"] = len(validators_df)
acps_support_df["networkTotalStake"] = int(avalanche_total_stake) / 1e9
acps_support_df["supportersPercentage"] = acps_support_df["supportersCount"] / acps_support_df["networkValidatorsCount"]
acps_support_df["objectorsPercentage"] = acps_support_df["objectorsCount"] / acps_support_df["networkValidatorsCount"]
acps_support_df["supportersStakePercentage"] = acps_support_df["supportersTotalStake"] / acps_support_df["networkTotalStake"]
acps_support_df["objectorsStakePercentage"] = acps_support_df["objectorsTotalStake"] / acps_support_df["networkTotalStake"]
acps_support_df["votesCount"] = acps_support_df["supportersCount"] + acps_support_df["objectorsCount"]
acps_support_df["votesPercentage"] = acps_support_df["votesCount"] / acps_support_df["networkValidatorsCount"]
acps_support_df["votingStake"] = acps_support_df["supportersTotalStake"] + acps_support_df["objectorsTotalStake"]
acps_support_df["votingStakePercentage"] = acps_support_df["votingStake"] / acps_support_df["networkTotalStake"]
acps_support_df["supportersVotesPercentage"] = acps_support_df["supportersCount"] / acps_support_df["votesCount"]
acps_support_df["objectorsVotesPercentage"] = acps_support_df["objectorsCount"] / acps_support_df["votesCount"]
acps_support_df["supportersVotingStakePercentage"] = acps_support_df["supportersTotalStake"] / acps_support_df["votingStake"]
acps_support_df["objectorsVotingStakePercentage"] = acps_support_df["objectorsTotalStake"] / acps_support_df["votingStake"]

acps_support_df = acps_support_df.fillna(0)

In [42]:
# ACPS supporters vs objectors bar chart (by number of validators)
acps_supporters_vs_objectors_number = px.bar(
	acps_support_df,
	x=["supportersCount", "objectorsCount"],
	y="acp",
	title="ACPs Supporters vs Objectors (by number of validators)",
	labels={"value": "Number of validators", "variable": "Type", "acp": "ACP"},
	color_discrete_sequence=["#00cc96", "#ef553b"],
	orientation="h",
	template="plotly_dark",
)

acps_supporters_vs_objectors_number.show()

In [43]:
# ACPS supporters vs objectors bar chart (by validator percentage)
acps_supporters_vs_objectors_percent = px.bar(
	acps_support_df,
	x=["supportersPercentage", "objectorsPercentage"],
	y="acp",
	title="ACPs Supporters vs Objectors (by validator percentage)",
	labels={"value": "Percentage of validators", "variable": "Type", "acp": "ACP"},
	color_discrete_sequence=["#00cc96", "#ef553b"],
	orientation="h",
	template="plotly_dark",
)
acps_supporters_vs_objectors_percent.update_layout(
	xaxis_tickformat="%"
)

acps_supporters_vs_objectors_percent.show()

In [44]:
# ACPS supporters vs objectors bar chart (by stake in AVAX)
# 1 AVAX = 10^9 nAVAX
acps_supporters_vs_objectors_stake = px.bar(
    acps_support_df,
    x=["supportersTotalStake", "objectorsTotalStake"],
    y="acp",
    title="ACPs Supporters vs Objectors (by stake in AVAX)",
    labels={"value": "Stake in AVAX", "variable": "Type", "acp": "ACP"},
    color_discrete_sequence=["#00cc96", "#ef553b"],
    orientation="h",
	template="plotly_dark",
)

acps_supporters_vs_objectors_stake.show()

In [45]:
# ACPS supporters vs objectors bar chart (by stake percentage)
acps_supporters_vs_objectors_stake_percent = px.bar(
	acps_support_df,
	x=["supportersStakePercentage", "objectorsStakePercentage"],
	y="acp",
	title="ACPs Supporters vs Objectors (by stake percentage)",
	labels={"value": "Stake percentage", "variable": "Type", "acp": "ACP"},
	color_discrete_sequence=["#00cc96", "#ef553b"],
	orientation="h",
	template="plotly_dark",
)
acps_supporters_vs_objectors_stake_percent.update_layout(
	xaxis_tickformat="%",
)

acps_supporters_vs_objectors_stake_percent.show()

In [28]:
# Push the charts to Plotly Chart Studio
import chart_studio.plotly as py

py.plot(acps_table, filename="acps-table", auto_open=False)
py.plot(acps_by_status, filename="acps-by-status", auto_open=False)
py.plot(acps_supporters_vs_objectors_number, filename="acps-supporters-vs-objectors-number", auto_open=False)
py.plot(acps_supporters_vs_objectors_percent, filename="acps-supporters-vs-objectors-percent", auto_open=False)
py.plot(acps_supporters_vs_objectors_stake, filename="acps-supporters-vs-objectors-stake", auto_open=False)
py.plot(acps_supporters_vs_objectors_stake_percent, filename="acps-supporters-vs-objectors-stake-percentage", auto_open=False)

'https://plotly.com/~Nuttymoon/16/'

In [46]:
# Save the tables to CSV as one table
acps_table_df = (
	acps_df
	.merge(acps_support_df, on="acp", how="left")
)

acps_table_df.to_csv("output/acps-support.csv", index=False)